In [2]:
!pip3 install findspark

In [5]:
import findspark
findspark.init()

In [8]:
import os
import pyspark

In [7]:
#executing the shell.py inside the 'SPARK_HOME' environment

exec(open(os.path.join(os.environ["SPARK_HOME"],'python/pyspark/shell.py')).read())

ERROR StatusLogger Reconfiguration failed: No configuration found for '67424e82' at 'null' in 'null'
ERROR StatusLogger Reconfiguration failed: No configuration found for 'Default' at 'null' in 'null'
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/04 18:18:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.3.2
      /_/

Using Python version 3.9.13 (main, Aug 25 2022 18:29:29)
Spark context Web UI available at http://192.168.0.25:4040
Spark context available as 'sc' (master = local[*], app id = local-1691183893816).
SparkSession available as 'spark'.


In [10]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark.read.json('/Users/tania/Downloads/spark/Spark-Programming-In-Python-master/notebook/data/people.json').show()

+----+--------+
| age|    name|
+----+--------+
|null|Prashant|
|  30|   Abdul|
|  19|  Justin|
|  43|    Andy|
+----+--------+

